# SQLModel

SQLModel is a library for interacting with SQL databases in Python. It is designed to be intuitive and easy to use. It is powered by [Pydantic](https://docs.pydantic.dev/latest/) and [SQLAlchemy](https://sqlalchemy.org/).

It combines SQLAlchemy and Pydantic and tries to simplify the code you write as much as possible, allowing you to reduce the code duplication to a minimum, but while getting the best developer experience possible.

In [1]:
from typing import Optional

from sqlmodel import Field, SQLModel

class Hero(SQLModel, table=True):
    id: int | None = Field(default=None, primary_key=True)
    name: str
    secret_name: str
    age: int | None

The `id` field can be `None` in Python code before interacting with the database, but can not be `null` in the database. Once we call `session.commit()`, the the `id` will be assigned in the database and the objects in our code will **expire**. Expiration is a protection from holding in memory outdated values. Next time we request any field, for example `.name`, SQLAlchemy will make sure to contact the database and **get the most recent version if the data** (updating that field name in our object and then making it available for the rest of the Python expression.).

In [2]:
hero_1 = Hero(name="Deadpond", secret_name="Dive Wilson")
hero_2 = Hero(name="Spider", secret_name="Peter Parkour", age=18)
hero_3 = Hero(name="Rusty", secret_name="Tommy Sharp", age=48)

Writing to the database

In [3]:
import os
from sqlmodel import create_engine, Session

engine = create_engine("sqlite:///:memory:")
SQLModel.metadata.create_all(engine)

with Session(engine) as session:
    session.add(hero_1)
    session.add(hero_2)
    session.add(hero_3)
    session.commit()

    print(f"SQLAlchemy has no way to recognize an attempt to query an object if I don't access a filed: {hero_1}")
    print(f"However, once I query any field, it will implicitly refresh data: {hero_1.id} ||| {hero_1}")

    session.refresh(hero_2)

print(f"The alternative is to refresh the object explicitly: {hero_2}")  # Yes, it is fine to access the object after the session ends.

SQLAlchemy has no way to recognize an attempt to query an object if I don't access a filed: 
However, once I query any field, it will implicitly refresh data: 1 ||| age=None secret_name='Dive Wilson' name='Deadpond' id=1
The alternative is to refresh the object explicitly: age=18 name='Spider' secret_name='Peter Parkour' id=2


 # Upsert

Typical upsert operation:

```python
def upsert_device(device_uuid: str, device: Device) -> Device:
    with Session(engine) as session:
        # check if the device exists
        statement = select(Device).where(Device.uuid == device_uuid)
        result = session.exec(statement).first()

        # if not, create it
        if result is None:
            result = device

        # sync the data
        for key, value in device.dict(exclude_unset=True).items():
            setattr(result, key, value)

        # persist the data to the database
        session.add(result)
        session.commit()
        session.refresh(result)

        return result
```

# Read Data

Reading from the database. `select` returns a specific type of an object. It can be used with any number of `where` methods as well as some others methods.

In [19]:
from sqlmodel import select, or_

with Session(engine) as session:
    statement = select(Hero).where(Hero.age).where(Hero.secret_name.like("%Peter%"))
    # Multiple where clauses are also fine:
    # statement = select(Hero).where(Hero.age > 20 Hero.age < 50)
    # OR also supported:
    # from sqlmodel import select, or_
    # statement = select(Hero).where(or_(Hero.age <= 35, Hero.age > 90))
    heros = session.exec(statement)
    
    print("iterator object:", heros)
    for hero in heros:
        print(f"Hero: {hero.name}, age: {hero.age}")

    print(f"All objects: {session.exec(statement).all()}")
    

iterator object: <sqlalchemy.engine.result.ScalarResult object at 0x113f25540>
Hero: Spider, age: 18
All objects: [Hero(age=18, name='Spider', secret_name='Peter Parkour', id=2)]


Sometimes autocomplete can complain about `Optional` attributes. It can be fixed with `from sqlmodel import col`:

```python
    with Session(engine) as session:
        statement = select(Hero).where(col(Hero.age) >= 35)
        results = session.exec(statement)
        for hero in results:
            print(hero)
```